<a href="https://colab.research.google.com/github/jasonfuk/hello-world/blob/master/MyhW2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
!pip3 install numpy scipy pandas matplotlib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn import tree
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn import metrics
# visualization
import seaborn as sns
import matplotlib.pyplot as plt

Mounted at /content/gdrive
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
myTrain=pd.read_csv('/content/gdrive/MyDrive/MyDATAHW/train.csv', index_col=None)
myTest=pd.read_csv('/content/gdrive/MyDrive/MyDATAHW/test.csv', index_col=None)


In [ ]:
X = myTrain.iloc[:,:-1]
Y = myTrain.iloc[:,-1]

In [ ]:
X.shape

(17103, 16)

In [ ]:
Y.shape

(17103,)

In [ ]:
X.isnull().mean()

Attribute1     0.000000
Attribute2     0.000000
Attribute3     0.004151
Attribute4     0.001871
Attribute5     0.009414
Attribute6     0.432731
Attribute7     0.481261
Attribute8     0.063907
Attribute9     0.063498
Attribute10    0.025259
Attribute11    0.017892
Attribute12    0.024265
Attribute13    0.097585
Attribute14    0.406478
Attribute15    0.017190
Attribute16    0.009414
dtype: float64

In [ ]:
Y.isnull().sum()

0

In [ ]:
np.unique(Y)

array(['No', 'Yes'], dtype=object)

In [ ]:
#分訓練資料和測試資料
from sklearn.model_selection import train_test_split
Xtrain, Xtest, Ytrain, Ytest = train_test_split(X,Y,test_size=0.3,random_state=420) #隨機抽取

In [ ]:
Xtrain.shape

(11972, 16)

In [ ]:
Xtrain.head()

,Attribute1,Attribute2,Attribute3,Attribute4,Attribute5,Attribute6,Attribute7,Attribute8,Attribute9,Attribute10,Attribute11,Attribute12,Attribute13,Attribute14,Attribute15,Attribute16
6414,2016-03-24,23,8.1,13.0,0.0,NaN,NaN,NW,28.0,NW,11.0,64.0,NaN,NaN,12.5,No
13698,2014-11-22,29,13.8,25.6,0.0,NaN,8.9,WSW,43.0,SW,26.0,49.0,1013.4,6.0,22.4,No
12310,2016-08-07,28,1.6,17.4,0.0,1.2,9.3,NNE,31.0,NNW,20.0,36.0,1021.7,NaN,16.6,No
1072,2016-02-05,10,19.6,32.4,0.0,11.6,NaN,E,54.0,E,20.0,27.0,1009.9,1.0,30.9,No
695,2016-01-16,4,13.2,22.7,0.6,NaN,NaN,SSE,43.0,ESE,15.0,74.0,1026.2,NaN,19.9,No


In [ ]:
for i in [Xtrain, Xtest, Ytrain, Ytest]:
    i.index = range(i.shape[0])

In [ ]:
for i in [myTest]:
    i.index = range(i.shape[0])

In [ ]:

myTest.shape

(806, 16)

In [ ]:
#查看样本是否不均衡
Ytrain.value_counts()  #发现样本不均衡

No     9751
Yes    2221
Name: Attribute17, dtype: int64

In [ ]:
#對標籤編碼
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OrdinalEncoder
encorder = LabelEncoder().fit(Ytrain)
Ytrain = pd.DataFrame(encorder.transform(Ytrain))
Ytest = pd.DataFrame(encorder.transform(Ytest))

In [ ]:
encorder.classes_# 查看有几分类


array(['No', 'Yes'], dtype=object)

In [ ]:
Ytrain.head()

,0
0,0
1,0
2,0
3,0
4,0


In [ ]:
#对特征矩阵进行处理
#描述性统计
#describe只能对数值型数据进行描述
Xtrain.describe([0.01,0.05,0.1,0.25,0.5,0.75,0.9,0.99]).T

,count,mean,std,min,1%,5%,10%,25%,50%,75%,90%,99%,max
Attribute2,11972.0,23.669312,14.245857,0.0,0.000,2.0,4.0,11.0,23.0,36.0,44.0,48.000,48.0
Attribute3,11914.0,12.115100,6.375230,-8.0,-1.887,1.7,3.8,7.5,12.0,16.8,20.6,25.600,31.0
Attribute4,11945.0,23.355463,7.110431,-2.9,9.388,13.0,14.6,18.0,22.8,28.5,33.0,40.056,47.0
Attribute5,11854.0,2.161515,7.592546,0.0,0.000,0.0,0.0,0.0,0.0,0.6,5.6,36.294,147.2
Attribute6,6813.0,5.522252,4.174146,0.0,0.400,1.0,1.4,2.6,4.8,7.4,10.2,17.776,81.2
Attribute7,6222.0,7.816361,3.717399,0.0,0.000,0.5,1.8,5.2,8.7,10.7,12.1,13.400,14.3
Attribute9,11217.0,39.656860,13.393549,9.0,15.000,20.0,24.0,31.0,39.0,46.0,57.0,80.000,109.0
Attribute11,11751.0,18.553825,8.696391,0.0,2.000,6.0,9.0,13.0,19.0,24.0,30.0,43.000,72.0
Attribute12,11683.0,50.347342,20.458698,1.0,8.000,16.0,22.0,36.0,51.0,64.0,77.0,97.000,100.0
Attribute13,10783.0,1015.459779,7.034767,977.1,998.882,1004.4,1006.7,1010.6,1015.5,1020.2,1024.4,1031.618,1037.9


In [ ]:
col = Xtrain.mean().index

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  """Entry point for launching an IPython kernel.


In [ ]:
Xtest.describe([0.01,0.05,0.1,0.25,0.5,0.75,0.9,0.99]).T


,count,mean,std,min,1%,5%,10%,25%,50%,75%,90%,99%,max
Attribute2,5131.0,23.803937,14.550073,0.0,0.0,2.0,4.0,11.0,24.00,36.00,44.00,48.000,48.0
Attribute3,5118.0,12.044256,6.377130,-8.5,-1.9,1.8,3.8,7.5,11.90,16.60,20.53,25.800,29.9
Attribute4,5126.0,23.240051,7.045357,0.1,9.7,12.9,14.5,18.1,22.50,28.10,32.70,40.000,46.5
Attribute5,5088.0,2.167217,8.276109,0.0,0.0,0.0,0.0,0.0,0.00,0.60,5.20,33.052,156.0
Attribute6,2889.0,5.463274,4.474168,0.0,0.4,0.8,1.4,2.6,4.60,7.20,10.20,19.048,77.3
Attribute7,2650.0,7.719396,3.691860,0.0,0.0,0.6,1.7,5.1,8.55,10.70,12.00,13.400,14.0
Attribute9,4800.0,39.462500,13.565345,7.0,15.0,20.0,24.0,30.0,37.00,46.00,57.00,81.000,126.0
Attribute11,5046.0,18.492073,8.782576,0.0,2.0,6.0,7.0,13.0,17.00,24.00,30.00,43.000,72.0
Attribute12,5005.0,50.659341,20.300226,1.0,8.0,17.0,23.0,36.0,51.00,65.00,77.00,96.000,100.0
Attribute13,4651.0,1015.578284,6.964499,982.2,998.9,1004.4,1006.9,1010.9,1015.50,1020.30,1024.50,1032.100,1038.9


In [ ]:
# Attribute1是當天日期，我選擇drop該col
Xtrain.drop(['Attribute1'],axis = 1,inplace = True)

KeyError: ignored

In [ ]:
# Attribute2是當天日期，我選擇drop該col
Xtrain.drop(['Attribute2'],axis = 1,inplace = True)

In [ ]:
Xtest.drop(['Attribute1'],axis = 1,inplace = True)

In [ ]:
Xtest.drop(['Attribute2'],axis = 1,inplace = True)

In [ ]:
myTest.drop(['Attribute2','Attribute1'],axis = 1,inplace = True)

In [ ]:
myTest.head()

,Attribute3,Attribute4,Attribute5,Attribute6,Attribute7,Attribute8,Attribute9,Attribute10,Attribute11,Attribute12,Attribute13,Attribute14,Attribute15,Attribute16
0,10.5,19.5,0.2,3.4,2.9,NNE,28.0,WNW,7.0,56.0,1011.5,8.0,16.8,No
1,8.7,17.7,0.0,13.0,9.7,WSW,63.0,W,31.0,24.0,1011.4,5.0,16.3,No
2,17.5,33.5,0.0,13.6,12.5,WSW,39.0,NW,9.0,13.0,1012.0,1.0,30.7,No
3,18.5,30.3,0.0,8.6,11.4,S,28.0,S,13.0,18.0,1017.5,1.0,28.8,No
4,9.4,13.3,4.6,1.8,0.0,SW,39.0,W,11.0,83.0,1007.9,7.0,13.2,Yes


In [ ]:
# Attribute5為降雨量，如果大於1當日降雨設為Yes
Xtrain.loc[Xtrain["Attribute5"] >= 1,"Attribute16"] = "Yes"
Xtrain.loc[Xtrain["Attribute5"] < 1,"Attribute16"] = "No"
Xtrain.loc[Xtrain["Attribute5"] == np.nan,"Attribute16"] = np.nan

In [ ]:
Xtrain.head()

,Attribute2,Attribute3,Attribute4,Attribute5,Attribute6,Attribute7,Attribute8,Attribute9,Attribute10,Attribute11,Attribute12,Attribute13,Attribute14,Attribute15,Attribute16
0,23,8.1,13.0,0.0,NaN,NaN,NW,28.0,NW,11.0,64.0,NaN,NaN,12.5,No
1,29,13.8,25.6,0.0,NaN,8.9,WSW,43.0,SW,26.0,49.0,1013.4,6.0,22.4,No
2,28,1.6,17.4,0.0,1.2,9.3,NNE,31.0,NNW,20.0,36.0,1021.7,NaN,16.6,No
3,10,19.6,32.4,0.0,11.6,NaN,E,54.0,E,20.0,27.0,1009.9,1.0,30.9,No
4,4,13.2,22.7,0.6,NaN,NaN,SSE,43.0,ESE,15.0,74.0,1026.2,NaN,19.9,No


In [ ]:
Xtest.loc[Xtrain["Attribute5"] >= 1,"Attribute16"] = "Yes"
Xtest.loc[Xtrain["Attribute5"] < 1,"Attribute16"] = "No"
Xtest.loc[Xtrain["Attribute5"] == np.nan,"Attribute16"] = np.nan

In [ ]:
cate = Xtrain.columns[Xtrain.dtypes == "object"].tolist()


In [ ]:
cate

['Attribute8', 'Attribute10', 'Attribute16']

In [ ]:
cloud = ["Attribute14"]

In [ ]:
cate = cate + cloud

In [ ]:
cate

['Attribute8', 'Attribute10', 'Attribute16', 'Attribute14']

In [ ]:
from sklearn.impute import SimpleImputer #0.20, conda, pip

si = SimpleImputer(missing_values=np.nan,strategy="most_frequent")
#注意，我们使用训练集数据来训练我们的填补器，本质是在生成训练集中的众数
si.fit(Xtrain.loc[:,cate])

#然后我们用训练集中的众数来同时填补训练集和测试集
Xtrain.loc[:,cate] = si.transform(Xtrain.loc[:,cate])
Xtest.loc[:,cate] = si.transform(Xtest.loc[:,cate])

In [ ]:
myTest.loc[:,cate] = si.transform(myTest.loc[:,cate])

In [ ]:
Xtrain.loc[:,cate].isnull().mean()

Attribute8     0.0
Attribute10    0.0
Attribute16    0.0
Attribute14    0.0
dtype: float64

In [ ]:
Xtest.loc[:,cate].isnull().mean()


Attribute8     0.0
Attribute10    0.0
Attribute16    0.0
Attribute14    0.0
dtype: float64

In [ ]:
myTest.loc[:,cate].isnull().mean()

Attribute8     0.0
Attribute10    0.0
Attribute16    0.0
Attribute14    0.0
dtype: float64

In [ ]:
#将所有的分类型变量编码为数字，一个类别是一个数字
from sklearn.preprocessing import OrdinalEncoder #只允许二维以上的数据进行输入
oe = OrdinalEncoder()

#利用训练集进行fit
oe = oe.fit(Xtrain.loc[:,cate])

#用训练集的编码结果来编码训练和测试特征矩阵
#在这里如果测试特征矩阵报错，就说明测试集中出现了训练集中从未见过的类别
Xtrain.loc[:,cate] = oe.transform(Xtrain.loc[:,cate])
Xtest.loc[:,cate] = oe.transform(Xtest.loc[:,cate])

In [ ]:
myTest.loc[:,cate] = oe.transform(myTest.loc[:,cate])

In [ ]:
Xtrain.loc[:,cate].head()

,Attribute8,Attribute10,Attribute16,Attribute14
0,7.0,7.0,0.0,7.0
1,15.0,12.0,0.0,6.0
2,5.0,6.0,0.0,7.0
3,0.0,0.0,0.0,1.0
4,10.0,2.0,0.0,7.0


In [ ]:
myTest.loc[:,cate].head()

,Attribute8,Attribute10,Attribute16,Attribute14
0,5.0,14.0,0.0,8.0
1,15.0,13.0,0.0,5.0
2,15.0,7.0,0.0,1.0
3,8.0,8.0,0.0,1.0
4,12.0,13.0,1.0,7.0


In [ ]:
col = Xtrain.columns.tolist()

print(col)

print(cate)

['Attribute3', 'Attribute4', 'Attribute5', 'Attribute6', 'Attribute7', 'Attribute8', 'Attribute9', 'Attribute10', 'Attribute11', 'Attribute12', 'Attribute13', 'Attribute14', 'Attribute15', 'Attribute16']
['Attribute8', 'Attribute10', 'Attribute16', 'Attribute14']


In [ ]:
for i in cate:
    col.remove(i)

In [ ]:
col

['Attribute3',
 'Attribute4',
 'Attribute5',
 'Attribute6',
 'Attribute7',
 'Attribute9',
 'Attribute11',
 'Attribute12',
 'Attribute13',
 'Attribute15']

In [ ]:
#实例化模型，填补策略为"mean"表示均值
impmean = SimpleImputer(missing_values=np.nan,strategy = "mean")
#用训练集来fit模型
impmean = impmean.fit(Xtrain.loc[:,col])
#分别在训练集和测试集上进行均值填补
Xtrain.loc[:,col] = impmean.transform(Xtrain.loc[:,col])
Xtest.loc[:,col] = impmean.transform(Xtest.loc[:,col])

In [ ]:
myTest.loc[:,col] = impmean.transform(myTest.loc[:,col])

In [ ]:
Xtrain.isnull().mean()

Attribute3     0.0
Attribute4     0.0
Attribute5     0.0
Attribute6     0.0
Attribute7     0.0
Attribute8     0.0
Attribute9     0.0
Attribute10    0.0
Attribute11    0.0
Attribute12    0.0
Attribute13    0.0
Attribute14    0.0
Attribute15    0.0
Attribute16    0.0
dtype: float64

In [ ]:
Xtest.isnull().mean()

Attribute3     0.0
Attribute4     0.0
Attribute5     0.0
Attribute6     0.0
Attribute7     0.0
Attribute8     0.0
Attribute9     0.0
Attribute10    0.0
Attribute11    0.0
Attribute12    0.0
Attribute13    0.0
Attribute14    0.0
Attribute15    0.0
Attribute16    0.0
dtype: float64

In [ ]:
myTest.isnull().mean()

Attribute3     0.0
Attribute4     0.0
Attribute5     0.0
Attribute6     0.0
Attribute7     0.0
Attribute8     0.0
Attribute9     0.0
Attribute10    0.0
Attribute11    0.0
Attribute12    0.0
Attribute13    0.0
Attribute14    0.0
Attribute15    0.0
Attribute16    0.0
dtype: float64

In [ ]:
from sklearn.preprocessing import StandardScaler #数据转换为均值为0，方差为1的数据
#标准化不改变数据的分布，不会把数据变成正态分布的

In [ ]:
ss = StandardScaler()
ss = ss.fit(Xtrain.loc[:,col])
Xtrain.loc[:,col] = ss.transform(Xtrain.loc[:,col])
Xtest.loc[:,col] = ss.transform(Xtest.loc[:,col])

In [ ]:
myTest.loc[:,col] = ss.transform(myTest.loc[:,col])

In [ ]:
Xtrain.head()

,Attribute3,Attribute4,Attribute5,Attribute6,Attribute7,Attribute8,Attribute9,Attribute10,Attribute11,Attribute12,Attribute13,Attribute14,Attribute15,Attribute16
0,-0.631355,-1.458082,-0.286115,0.000000,3.314472e-16,7.0,-0.899188,7.0,-0.876783,0.675560,-1.702920e-14,7.0,-1.358864,0.0
1,0.264942,0.316038,-0.286115,0.000000,4.043883e-01,15.0,0.257883,12.0,0.864288,-0.066669,-3.085351e-01,6.0,0.079556,0.0
2,-1.653447,-0.838548,-0.286115,-1.372741,5.536589e-01,5.0,-0.667773,6.0,0.167860,-0.709934,9.347251e-01,7.0,-0.763154,0.0
3,1.176963,1.273500,-0.286115,1.930284,3.314472e-16,0.0,1.106402,0.0,0.167860,-1.155272,-8.328015e-01,1.0,1.314564,0.0
4,0.170595,-0.092291,-0.206694,0.000000,3.314472e-16,10.0,0.257883,2.0,-0.412498,1.170379,1.608782e+00,7.0,-0.283681,0.0


In [ ]:
Ytrain.head()

,0
0,0
1,0
2,0
3,0
4,0


In [ ]:
from time import time #随时监控我们的模型的运行时间
import datetime
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score
from sklearn.metrics import roc_auc_score, recall_score

In [ ]:
Ytrain = Ytrain.iloc[:,0].ravel()
Ytest = Ytest.iloc[:,0].ravel()

In [ ]:
times = time()
for kernel in ["linear","poly","rbf","sigmoid"]:
    clf = SVC(kernel = kernel
              ,gamma="auto"
              ,degree = 1
              ,cache_size = 5000
              ,class_weight = "balanced"
             ).fit(Xtrain, Ytrain)
    result = clf.predict(myTest)
    score = clf.score(Xtest,Ytest)
    recall = recall_score(Ytest, result)
    auc = roc_auc_score(Ytest,clf.decision_function(Xtest))
    print("%s 's testing accuracy %f, recall is %f', auc is %f" % (kernel,score,recall,auc))
    print(datetime.datetime.fromtimestamp(time()-times).strftime("%M:%S:%f"))

ValueError: ignored

In [ ]:
result

array([1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1,
       1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1,
       1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0,
       1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0,
       0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1,
       1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1,
       1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0,
       0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0,
       0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0,
       1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1,
       1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1,
       1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1,

In [ ]:
import csv
with open('test.csv', 'x') as f: 
    f.write('id,ans\n') 
    for i in range(len(result)): 
        f.write(str(float(i)) + ',' + str(int(result[i])) + '\n')